Creates Tensorflow Graphs for spark-nlp DL Annotators and Models


In [ ]:
import tensorflow as tf

from ner_model import NerModel

In [ ]:
def create_graph(ntags, embeddings_dim, nchars, chars_dim, char_filters, lstm_size = 200):

    tf.reset_default_graph()
    model_name = 'char_cnn_blstm_{}_{}_{}_{}_{}'.format(ntags, embeddings_dim, nchars, chars_dim, char_filters)
    with tf.Session() as session:
        ner = NerModel(session=None)
        ner.add_cnn_char_repr(nchars, chars_dim, char_filters)
        ner.add_pretrained_word_embeddings(embeddings_dim)
        ner.add_context_repr(ntags, lstm_size)
        ner.add_inference_layer(False)
        ner.add_training_op(5)
        
        ner.init_variables()
        
        saver = tf.train.Saver()
    
        file_name = model_name + '.pb'
        tf.train.write_graph(ner.session.graph, './', file_name, False)
        ner.close()
        session.close()

In [ ]:
create_graph(10, 200, 200, 25, 30)